<a href="https://colab.research.google.com/github/Ucchwas/Detect-Arrhythmia-from-training2017-Dataset/blob/main/Detect_Arrhythmia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import os
import zipfile

# Download the dataset
url = "https://physionet.org/files/challenge-2017/1.0.0/training2017.zip"
filename = "training2017.zip"
if not os.path.exists(filename):
    urllib.request.urlretrieve(url, filename)

# Extract the dataset
with zipfile.ZipFile(filename, "r") as zip_ref:
    zip_ref.extractall(".")


In [21]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
import os

print(os.path.isfile("training2017/REFERENCE.csv"))

True


In [23]:
# Define the signal length (in samples) to use
signal_length = 3000

# Load the training dataset
train_data_path = "training2017"
train_labels = pd.read_csv(os.path.join(train_data_path, "REFERENCE.csv"), header=None, names=["filename", "label"])
train_labels["filename"] = train_labels["filename"].apply(lambda x: os.path.splitext(x)[0])
train_data = np.zeros((len(train_labels), signal_length))

In [27]:
train_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
for i, row in train_labels.iterrows():
    mat = loadmat(os.path.join(train_data_path, row["filename"]))
    signal = mat["val"].reshape(-1)
    # Adjust the signal length to match X_train
    if len(signal) < signal_length:
        signal = np.pad(signal, (0, signal_length - len(signal)), 'constant')
    elif len(signal) > signal_length:
        signal = signal[:signal_length]
    signal = StandardScaler().fit_transform(signal.reshape(-1, 1)).reshape(-1)
    train_data[i] = signal

In [30]:
# Convert the labels to integers
le = LabelEncoder()
train_labels["label_num"] = le.fit_transform(train_labels["label"])

In [32]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels["label_num"], test_size=0.2, random_state=42)

In [33]:
# Define the model architecture
model = Sequential([
    Dense(256, activation="relu", input_shape=(signal_length,)),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(4, activation="softmax")
])

In [34]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, callbacks=[early_stop])

Epoch 1/50
214/214 [==============================] - 5s 16ms/step - loss: 1.4366 - accuracy: 0.4733 - val_loss: 1.0586 - val_accuracy: 0.6120
Epoch 2/50
214/214 [==============================] - 4s 19ms/step - loss: 1.1191 - accuracy: 0.5447 - val_loss: 1.0541 - val_accuracy: 0.6149
Epoch 3/50
214/214 [==============================] - 4s 17ms/step - loss: 1.0689 - accuracy: 0.5699 - val_loss: 1.0502 - val_accuracy: 0.6149
Epoch 4/50
214/214 [==============================] - 4s 19ms/step - loss: 1.0546 - accuracy: 0.5797 - val_loss: 1.0312 - val_accuracy: 0.6149
Epoch 5/50
214/214 [==============================] - 4s 18ms/step - loss: 1.0452 - accuracy: 0.5812 - val_loss: 1.0081 - val_accuracy: 0.6149
Epoch 6/50
214/214 [==============================] - 3s 16ms/step - loss: 1.0374 - accuracy: 0.5849 - val_loss: 1.0009 - val_accuracy: 0.6149
Epoch 7/50
214/214 [==============================] - 3s 15ms/step - loss: 1.0228 - accuracy: 0.5862 - val_loss: 1.0128 - val_accuracy: 0.6149

In [36]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(X_val, y_val)
print("Validation loss:", val_loss)
print("Validation accuracy:", val_acc)


54/54 [==============================] - 0s 4ms/step - loss: 0.9750 - accuracy: 0.6149
Validation loss: 0.9750270247459412
Validation accuracy: 0.614888608455658
